# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [1]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [2]:
nutrition_agent = Agent(
    name="Nutrition_Assistant",
    instructions="""
    you are a helpful assisstant giving nutrition advise. 
    you give concise answers
    """
    )

Let's execute the Agent:

In [3]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition_Assistant", ...)
- Final output (str):
    Bananas are healthy—nutritious and convenient. Key points:
    
    - Good: potassium (supports blood pressure), vitamin B6, vitamin C, fiber; low in fat.
    - Energy: about 100 calories per medium banana; natural sugars, so portion matters if tracking sugar intake.
    - Consider: ripe = more sugars; unripe = more resistant starch (may aid gut health).
    - mind kidney health: people with kidney disease may need to limit potassium.
    - overall: great as part of a balanced diet (e.g., with yogurt, oats, or peanut butter).
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [4]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are quite healthy as part of a balanced diet.

Key nutrients (per medium banana, ~118 g):
- Potassium: supports blood pressure and heart function
- Vitamin B6, Vitamin C, dietary fiber
- Moderate calories (about 105 kcal)
- 14 g sugar and 3 g fiber; low fat

Benefits:
- Quick energy from carbs
- Aid digestion and heart health
- Convenient, portable snack

Things to consider:
- Higher glycemic index when very ripe; portion if managing blood sugar
- Kidney issues: limit if advised by a doctor due to potassium

Serving ideas: eat plain, with peanut butter, in smoothies, or sliced on cereal.

_Good Job!_